In [1]:
import os
import requests
from bs4 import BeautifulSoup
from flask import Flask, request, abort
from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.messaging import Configuration, ApiClient, MessagingApi, ReplyMessageRequest, TextMessage
from linebot.v3.webhooks import MessageEvent, TextMessageContent

line_access_token = os.environ.get('LINE_ACCESS_TOKEN')
line_secret = os.environ.get('LINE_SECRET')
port = 5000

configuration = Configuration(access_token=line_access_token)
handler = WebhookHandler(line_secret)


In [2]:
ngrok_key = os.environ.get('NGROK_KEY')
port = 5000

In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token(ngrok_key)
tunnel = ngrok.connect(5000, name="linebot_tunnel")
print(tunnel.public_url)

https://5e64-2001-b011-1-34ec-245d-1941-8c8f-ca9b.ngrok-free.app


t=2024-06-01T03:05:13+0800 lvl=warn msg="failed to open private leg" id=0322f4de4354 privaddr=localhost:5000 err="dial tcp [::1]:5000: connectex: No connection could be made because the target machine actively refused it."
t=2024-06-01T03:05:17+0800 lvl=warn msg="failed to open private leg" id=0c7cfdcd65cf privaddr=localhost:5000 err="dial tcp [::1]:5000: connectex: No connection could be made because the target machine actively refused it."
t=2024-06-01T03:05:19+0800 lvl=warn msg="failed to open private leg" id=13e31990ab7d privaddr=localhost:5000 err="dial tcp [::1]:5000: connectex: No connection could be made because the target machine actively refused it."
t=2024-06-01T06:37:42+0800 lvl=eror msg="heartbeat timeout, terminating session" obj=tunnels.session obj=csess id=89acd379c8ab clientid=e574444a5bc0cfaf92bf55e66e0ef0b1
t=2024-06-01T06:37:42+0800 lvl=eror msg="session closed, starting reconnect loop" obj=tunnels.session obj=csess id=d77535e83cc7 err="session closed"
t=2024-06-01T

In [10]:
def get_latest_article():
    url = 'https://ccc.technews.tw/'
    response = requests.get(url)
    if response.status_code != 200:
        return "無法存取該網址。"
    else:
        soup = BeautifulSoup(response.text, 'html.parser')
        latest_article = soup.find('div', class_='content')
        if latest_article:
            title = latest_article.find('h1', class_='entry-title').text.strip()
            link = latest_article.find('a')['href']
            return f"{title}\n{link}"
        else:
            return "找不到最新文章。"


In [11]:
latest_info = get_latest_article()
print(latest_info)

狗狗幣本尊 Kabosu 過世！迷因始祖走了 $DOGE 劇烈波動
https://ccc.technews.tw/2024/05/24/kabosu-the-prototype-of-the-dogecoin-meme-pass-away/


In [12]:
app = Flask(__name__)

@app.route("/", methods=['POST'])
def callback():
    signature = request.headers.get('X-Line-Signature', '')
    body = request.get_data(as_text=True)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    text = event.message.text
    if text == "3C新聞":
        latest_info = get_latest_article()
        with ApiClient(configuration) as api_client:
            line_bot_api = MessagingApi(api_client)
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=latest_info)]
                )
            )


if __name__ == "__main__":
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


In [38]:
pip install requests beautifulsoup4


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: requests in c:\users\user\anaconda3\envs\linebot\lib\site-packages (2.31.0)

